In [ ]:
# Qs 1.1
 # We have used pydub for manipulating audio files 
 # pip install pydub

from pydub import AudioSegment
from pydub.utils import make_chunks
import sys
import os
base_path = os.getcwd()


genre = "Full-On"  # Input name of the genre you want to preprocess

path = base_path+"\\"+genre

if os.path.exists(path)== True:       
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.wav' in file:
                files.append(os.path.join(r, file))
    i=0
    dirc = path + "\\chunks"
    if not os.path.exists(dirc):
            os.makedirs(dirc)
    for f in files:
        
        print(f)
        myaudio = AudioSegment.from_file(f , "wav") 
        chunk_length_ms = 30000 # pydub calculates in millisec
        chunks = make_chunks(myaudio, chunk_length_ms) # Make chunks of 30 secs
        
        for ind, chunk in enumerate(chunks):
            
            chunk_name = dirc + "\chunk{0}.wav".format(i)
            print(chunk_name)
            chunk.export(chunk_name, format="wav")
            i = i+1
else:
    print("Directory doesn't exist!") # If genre doesn't exist.

In [ ]:
# Qs 1.2.1 Standard features using existing libraries :

#pip install librosa

import sys
import os
import librosa
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing

base_path = os.getcwd()
genre = "Hi_Tech"  # Input the name of genre that you want to extract features from.


path = base_path+"\\"+ genre+ "\\chunks"
print(path)

if os.path.exists(path)== True:

    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.wav' in file:
                files.append(os.path.join(r, file))
    i = 0
    file_names, Mfcc, Spec_BW, Spec_cnt, Spec_roll, Mel_spec = [], [], [], [], [], []
    for f in files:
        
        print(i)
        y, sr = librosa.load(f)
        
        mfcc_ = librosa.feature.mfcc(y=y, sr=sr)    # Feature 1 Mel Frequency Cepstral Coeff
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)  # Feature 2 Mel Spectogram
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)    # Feature 3 Spectral bandwidth
        spec_cnt = librosa.feature.spectral_centroid(y=y, sr=sr)    # Feature 4 Spectral Centroid
        spec_roll = librosa.feature.spectral_rolloff(y=y, sr=sr)     # Feature 5 Spectral Rolloff
        
        # Qs. 1.3 part 1 :
        # Normalizing features to zero mean and unit variance
        mfcc_norm =  preprocessing.scale(mfcc_)
        mel_spec_norm =  preprocessing.scale(mel_spec)
        mel_spec_norm =  preprocessing.scale(mel_spec)
        spec_bw_norm =  preprocessing.scale(spec_bw)
        spec_cnt_norm =  preprocessing.scale(spec_cnt)
        spec_roll_norm =  preprocessing.scale(spec_roll)
        
        # Appending features of individual chunks to create a list of 5 feature vectors 
        file_names.append(f)
        Mfcc.append(mfcc_norm)
        Mel_spec.append(mel_spec_norm)
        Spec_BW.append(spec_bw_norm)
        Spec_cnt.append(spec_cnt_norm)
        Spec_roll.append(spec_roll_norm)
        i=i+1
    
    # Qs. 1.2.3 
    # Storing the extracted features to a csv file
    df = pd.DataFrame(list(zip(file_names, Mfcc, Mel_spec, Spec_BW, Spec_cnt, Spec_roll)), 
               columns =['Filename', 'mfcc', "mel_spectogram","Spectral_bandwidth","Spectral_centroid","Spectral_rolloff"]) 
        
    df_name = genre + "-features"
    df.to_csv(df_name+".csv",index=False)   

    print("Features created for {Genre}".format(Genre = genre))
    
    
else:
    print("Directory doesn't exist!")
    
   



In [ ]:
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(mfcc_)
mfcc_norm =  preprocessing.scale(mfcc_)
scaled_df.mean(axis=0)

In [ ]:
# Qs 1.2.1 Kalman Filter for Beat Estimation :


In [ ]:
# Qs 1.2.2



In [6]:
# Qs 1.3 part 2, Dividing data into training and testing set (75:25)
import os
import sys
import numpy as np
import pandas as pd
df_name = ['Goa-features', 'Full-On-features', 'Dark_Forest-features', 'Hi_Tech-features']

for i in range(len(df_name)):  # Assuming the files exist since we created them in the previous segment
    train = []
    test = []
    df_read = pd.read_csv(df_name[i]+".csv", sep=',',header=None)
    df = df_read.values
    train, test = np.split(df, [int(0.75*len(df))])   # Splitting into 75:25 ratio
    # writing to csv files for train and test set. 
    pd.DataFrame(train).to_csv(df_name[i]+"-train.csv", header=False, index=False) 
    pd.DataFrame(test).to_csv(df_name[i]+"-test.csv", header=False, index=False)
#     np.savetxt(df_name[i]+"-train.csv", train, delimiter=",", fmt='%s')
#     np.savetxt(df_name[i]+"-test.csv", test, delimiter=",", fmt='%s')